In [1]:
import os

os.environ['http_proxy'] = "http://squid.cs.ait.ac.th:3128/"
os.environ['https_proxy'] = "http://squid.cs.ait.ac.th:3128/"

In [4]:
!pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116 --user

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     |████████████████████████████████| 1977.9 MB 672 bytes/s          ��█▉           | 1288.4 MB 77.0 MB/s eta 0:00:09
     |████████████████████████████████| 24.2 MB 15 kB/s              
     |████████████████████████████████| 4.2 MB 62.2 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [3]:
torch.cuda.is_available()

True

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

mnist_train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

fashion_train = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
fashion_test = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

fashion_train_loader = DataLoader(fashion_train, batch_size=64, shuffle=True)
fashion_test_loader = DataLoader(fashion_test, batch_size=64, shuffle=False)

In [26]:
class CNNModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3) # 28x28x1 -> 26x26x32 / 2 = 13x13x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)# 11x11x64 -> 5x5x64
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        
        x = x.view(-1, 64 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [8]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

def test(model, device, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

base_model = CNNModel().to(device)
optimizer = torch.optim.Adam(base_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [28]:
for epoch in range(1, 6):
    train(base_model, device, mnist_train_loader, optimizer, criterion, epoch)
    test(base_model, device, mnist_test_loader, criterion)

Train Epoch: 1 [0/60000] Loss: 2.298948
Train Epoch: 1 [6400/60000] Loss: 0.356841
Train Epoch: 1 [12800/60000] Loss: 0.360970
Train Epoch: 1 [19200/60000] Loss: 0.138276
Train Epoch: 1 [25600/60000] Loss: 0.225226
Train Epoch: 1 [32000/60000] Loss: 0.147682
Train Epoch: 1 [38400/60000] Loss: 0.069759
Train Epoch: 1 [44800/60000] Loss: 0.169684
Train Epoch: 1 [51200/60000] Loss: 0.045199
Train Epoch: 1 [57600/60000] Loss: 0.116820

Test set: Average loss: 0.0007, Accuracy: 9851/10000 (98.51%)

Train Epoch: 2 [0/60000] Loss: 0.111176
Train Epoch: 2 [6400/60000] Loss: 0.076573
Train Epoch: 2 [12800/60000] Loss: 0.093073
Train Epoch: 2 [19200/60000] Loss: 0.027703
Train Epoch: 2 [25600/60000] Loss: 0.104041
Train Epoch: 2 [32000/60000] Loss: 0.360858
Train Epoch: 2 [38400/60000] Loss: 0.153771
Train Epoch: 2 [44800/60000] Loss: 0.057919
Train Epoch: 2 [51200/60000] Loss: 0.017944
Train Epoch: 2 [57600/60000] Loss: 0.099698

Test set: Average loss: 0.0006, Accuracy: 9881/10000 (98.81%)

Tr

In [29]:
transfer_model = CNNModel().to(device)
transfer_model.load_state_dict(base_model.state_dict())

<All keys matched successfully>

In [31]:
for param in transfer_model.parameters():
    # print(param)
    param.requires_grad = False

In [34]:
transfer_model.fc2 = nn.Linear(128, 10, device=device)
transfer_model.fc2.requires_grad = True

In [ ]:
optimizer = torch.optim.Adam(transfer_model.fc2.parameters(), lr=0.001)

for epoch in range(1, 6):
    train(transfer_model, device, fashion_train_loader, optimizer, criterion, epoch)
    test(transfer_model, device, fashion_test_loader, criterion)
    
test_accuracy = test(transfer_model, device, fashion_test_loader, criterion)
print(f'Final Test Accuracy on Fashion MNIST: {test_accuracy:.2f}%')

Train Epoch: 1 [0/60000] Loss: 2.631480
Train Epoch: 1 [6400/60000] Loss: 1.386769
Train Epoch: 1 [12800/60000] Loss: 1.131358
Train Epoch: 1 [19200/60000] Loss: 1.324272
Train Epoch: 1 [25600/60000] Loss: 0.970456
Train Epoch: 1 [32000/60000] Loss: 1.211585
Train Epoch: 1 [38400/60000] Loss: 1.112245
Train Epoch: 1 [44800/60000] Loss: 1.074822
Train Epoch: 1 [51200/60000] Loss: 1.256624
Train Epoch: 1 [57600/60000] Loss: 1.088293

Test set: Average loss: 0.0153, Accuracy: 6852/10000 (68.52%)

Train Epoch: 2 [0/60000] Loss: 0.893758
Train Epoch: 2 [6400/60000] Loss: 1.107784
Train Epoch: 2 [12800/60000] Loss: 1.112038
Train Epoch: 2 [19200/60000] Loss: 1.207096
Train Epoch: 2 [25600/60000] Loss: 0.996716
Train Epoch: 2 [32000/60000] Loss: 0.967878
Train Epoch: 2 [38400/60000] Loss: 1.060265
Train Epoch: 2 [44800/60000] Loss: 1.056669
Train Epoch: 2 [51200/60000] Loss: 1.119566
Train Epoch: 2 [57600/60000] Loss: 1.296211

Test set: Average loss: 0.0145, Accuracy: 6984/10000 (69.84%)

Tr